TextRank구현

In [2]:
tokens = ["딸기","바나나","사과","딸기","파인애플"]
#unique한 값 뽑기 위해 set으로 중복 제거; unique한거 가져올때 set 많이 씀
# nodes = list(set(tokens))

vocab=['바나나','사과','파인애플','딸기'] #교재 순서와 맞추기 위해

#index가져와서 사전 만들기; vocab[i]:i ~~ 단어:인덱스
vocab2id = {vocab[i] : i for i in range(len(vocab))}
id2vocab = {i : vocab[i] for i in range(len(vocab))} #반대로도 하나 만들어줌

print(vocab2id)
print(id2vocab)

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}
{0: '바나나', 1: '사과', 2: '파인애플', 3: '딸기'}


In [34]:
#그래프 만들기: score만들기->빈 4*4 matrix 0으로 채움->동시등장하면 1씩 표시
import numpy as np

#weight matrix 만들기
vocab_len = len(vocab) #vocab_len에 맞는 행렬만들기 위함
weight_edge = np.zeros((vocab_len,vocab_len), dtype=np.float32)

#초기 score; 노드의 스코어 만들기
scores = np.ones(vocab_len, dtype=np.float32)

#vocab 순서대로 window 지나도록
window_size=2

covered_coocurrences = []

for window_start in range(len(tokens)-window_size+1): #range 특성 때문에 +1; window 개수 정의
    window = tokens[window_start : window_start + window_size] #start가 이동할 때마다 window 바꿔서 가져오기 위함
    for i in range(window_size -1):
        #startwindow부터 window 차례대로 돌기 위함; window_size만큼 반복
        #window_size가 2인 경우, range(0,1)~0번째만 돌아도 됨! ex. 딸기-바나나
        for j in range(i+1, window_size): #window에서 단어끼리 동시등장 비교하기 위한 반복문; 딸기-바나나, 딸기-사과
            
            if window[i] in vocab and window[j] in vocab: #window의 두 단어가 vocab에 있는 경우만 edge로 연결
                #이미 계산해서 edge 계산한 것들; 계산 반복 피하기 위함
                #start window0일 때 바나나:사과 = [1:2] ~> start window 1일 때 [0:1] ~so +1
                #그래서 원래 index값 찾기 위해 +1
                #window0 [2:3], window1[1,2], window2[0,1] ~~ window2의 경우, window_start가 0이므로 i=2를 더해줘서
                #--->원래의 인덱스 값 추출하도록!
                index_of_i = window_start + i
                index_of_j = window_start + j
                
                #weight_edge: 동시등장하는 단어관계에 +1 해주기
                if (index_of_i, index_of_j) not in covered_coocurrences:
                    weight_edge[vocab2id[window[i]]][vocab2id[window[j]]] = 1
                    weight_edge[vocab2id[window[j]]][vocab2id[window[i]]] = 1
                    #-->행렬 대칭이니까 여기도 +1
                    #-->겹치는 지점에 있는 애들은 이미 검사했으니까 그냥 넘어갈 수 있음
                    covered_coocurrences.append((index_of_i,index_of_j))

print(weight_edge)

#weight_matrix 만들기: 각 row의 sum구해서 그걸 edge 숫자에서 나눠줘야 함
for i in range(vocab_len):
    row_sum = weight_edge[i].sum()
    weight_edge[i] = weight_edge[i] / row_sum if row_sum > 0 else 0 #0으로 나누면 error나므로 이를 방지하기 위해
weight_edge #이게 교안p.36에 있는 weight_matrix -->이제 스코어에 element끼리 곱해서 더해야 함

[[0. 1. 0. 1.]
 [1. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 1. 1. 0.]]


array([[0.        , 0.5       , 0.        , 0.5       ],
       [0.5       , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333334, 0.33333334, 0.33333334, 0.        ]], dtype=float32)

In [47]:
#score값 업데이트
#위에서 구한 weight_edge와 scores를 element끼리 곱해서 더하기
MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001 #prev와 현재 차이가 이거보다 작으면 멈춤

for it in range(MAX_ITERATIONS): #사용하는 건x, 이 개수만큼 돌려주는 것
    prev_score = np.copy(scores) #prev_score 객체 만들어주기
    
    #numpy element-wise니까 단어 개수만큼 돌면 됨
    #weight_edge 열:i, 각 열과 score:j
    for i in range(vocab_len):
        
        summation = 0 #summation: 스코어랑 weight matrix 곱한 것들의 sum
        
        for j in range(vocab_len):
            summation += weight_edge[j][i]*prev_score[j]
            
        scores[i] = (1-d) + d * summation
    
    if np.sum(np.fabs(prev_score - scores)) <= threshold: #fabs:절대값
        break
scores

array([0.9837155 , 0.9837155 , 0.56564105, 1.4669281 ], dtype=float32)

In [48]:
#sorting
np.argsort(scores) #오름차순 sorting하고 해당 자리의 index 값을 반환

array([2, 0, 1, 3], dtype=int64)

In [50]:
#argsort의 결과(오름차순) 거꾸로(열 기준)
sorted_index = np.flip(np.argsort(scores),0)
sorted_index

array([3, 1, 0, 2], dtype=int64)

In [57]:
#키워드 score 보기
n = 3
for i in range(n):
    print("{} : {}".format(id2vocab[sorted_index[i]], scores[sorted_index[i]]))

딸기 : 1.4669281244277954
사과 : 0.9837154746055603
바나나 : 0.9837154746055603


TextRank로 뉴스 키워드 5개 끌어오기

In [115]:
def text_rank (tokens,vocab, window_size, num):

    vocab2id = {vocab[i] : i for i in range(len(vocab))}
    id2vocab = {i : vocab[i] for i in range(len(vocab))}


    vocab_len = len(vocab)

    weight_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

    score = np.ones(vocab_len,dtype=np.float32)

    coverd_coocurences = []
    
    for window_start in range(0,len(tokens)-window_size+1):
        window = tokens[window_start : window_start+window_size]
        
        for i in range(window_size-1):
            for j in range(i+1, window_size):            
                if window[i] in vocab and window[j] in vocab:
                    index_of_i = window_start + i 
                    index_of_j = window_start + j
                    
                    if (index_of_i,index_of_j) not in coverd_coocurences:
                        weight_edge[vocab2id[window[i]]][vocab2id[window[j]]] = 1
                        weight_edge[vocab2id[window[j]]][vocab2id[window[i]]] = 1
                        coverd_coocurences.append((index_of_i,index_of_j))

    for i in range(vocab_len):
        row_sum = weight_edge[i].sum() 
        weight_edge[i] = weight_edge[i]/row_sum if row_sum > 0 else 0
        
    MAX_ITERATIONS = 50
    d = 0.85
    threshold = 0.0001

    for it in range(MAX_ITERATIONS):
        prev_score = np.copy(score)

        for i in range(vocab_len):
            summation = 0
            for j in range(vocab_len):
                summation += weight_edge[j][i] * prev_score[j]

            score[i] = (1 - d) +d*summation

        if np.sum(np.fabs(prev_score - score)) <= threshold:
            break        

    sorted_index = np.flip(np.argsort(score),0)
    sorted_index            
    
    for i in range(num):
        print('{} {}'.format(id2vocab[sorted_index[i]], score[sorted_index[i]]))
        
        
for i,url in enumerate(urls):
    print(f'doc{i+1}')
    tokens = get_news(url).split(' ')
    vocab = list(set(get_news(url).split(' ')))
    
    text_rank(tokens,vocab,window_size=2,num=5)

doc1
데이터 10.217854499816895
센터 4.63233757019043
활용 3.913996934890747
계획 3.0218217372894287
경제 2.6802375316619873
doc2
데이터 3.142937421798706
서비스 2.401495933532715
한전 2.391519784927368
전력 2.309370279312134
제공 2.0044283866882324
doc3
경기도 3.9427449703216553
자율 3.9273698329925537
데이터 3.909980535507202
주행 3.5127453804016113
센터 2.4420278072357178
doc4
지역 4.432885646820068
인구 4.349888801574707
이동 3.6016430854797363
발생 2.8768599033355713
폭 2.7860758304595947
doc5
데이터 8.730308532714844
중계 6.474488735198975
기관 5.002007484436035
금융 4.417078495025635
인프라 3.601242780685425


강사님 풀이(함수 사용)

In [128]:
MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001

def get_scores(weight_edge, vocab_len):
    scores = np.ones(vocab_len, dtype=np.float32)
    
    for it in range(MAX_ITERATIONS):
        prev_score = np.copy(scores)

        for i in range(vocab_len):
            summation = (weight_edge[:, i] * prev_score).sum()
            scores[i] = (1 - d) + d * summation

        if np.sum(np.fabs(prev_score - scores)) <= threshold:
            break

    return scores

In [129]:
def get_weight_edge(vocab, tokens, window_size = 2):
    vocab_len = len(vocab)
    weight_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)
    covered_coocurrences = []

    for window_start in range(0, len(tokens)-window_size + 1):
        window = tokens[window_start : window_start + window_size]
        for i in range(window_size - 1):
            for j in range(i+1, window_size):

                if window[i] in vocab and window[j] in vocab:
                    index_of_i = window_start + i
                    index_of_j = window_start + j

                    if (index_of_i, index_of_j) not in covered_coocurrences:
                        weight_edge[vocab2id[window[i]]][vocab2id[window[j]]] = 1
                        weight_edge[vocab2id[window[j]]][vocab2id[window[i]]] = 1
                        covered_coocurrences.append((index_of_i, index_of_j))

    for i in range(vocab_len):
        row_sum =  weight_edge[i].sum()
        weight_edge[i] = weight_edge[i] / row_sum if row_sum > 0 else 0

    return weight_edge

In [130]:
def preprocess(news_article, node_pos= ["NNG", "NNP"]):
    
    vocab = [token[0] for token in mecab.pos(news_article) if token[1] in node_pos]
    tokens = [token[0] for token in mecab.pos(news_article)]
    
    vocab = list(set(vocab))
             
    vocab2id = { vocab[i] : i for i in range(len(vocab))}
    id2vocab = { i : vocab[i] for i in range(len(vocab))}
    
    vocab_len = len(vocab)
    
    return vocab, tokens, vocab2id, id2vocab, vocab_len

In [131]:
def report_result(scores, id2vocab):
    sorted_index = np.flip(np.argsort(scores), 0)
    
    print("핵심키워드")
    for i in range(top_n):        
        print("{} : {} ".format(id2vocab[sorted_index[i]], scores[sorted_index[i]]))
    print("")

In [132]:
import requests
from bs4 import BeautifulSoup

from konlpy.tag import Mecab
mecab = Mecab()


def get_news(url):
    headers = {"user-agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    html = BeautifulSoup(response.text, 'html.parser')
    
    article = html.select_one("#articleBodyContents").text
    return article

urls = ["https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191",
"https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728"]

docs = []
top_n = 5

for url in urls:
    news_article = get_news(url)
    
    vocab, tokens, vocab2id, id2vocab, vocab_len = preprocess(news_article)
    
    weight_edge = get_weight_edge(vocab, tokens, window_size=2)
    scores = get_scores(weight_edge, vocab_len)
    
    report_result(scores, id2vocab)

핵심키워드
데이터 : 7.4343390464782715 
활용 : 2.795384407043457 
정보 : 2.712031126022339 
플랫 : 2.188546657562256 
경제 : 2.1860430240631104 

핵심키워드
대회 : 1.6575965881347656 
한전 : 1.6287670135498047 
서비스 : 1.5968517065048218 
선정 : 1.4593236446380615 
전문 : 1.4593236446380615 

핵심키워드
주행 : 4.062406063079834 
데이터 : 3.5409369468688965 
자율 : 2.3876304626464844 
산업 : 1.9965639114379883 
뉴스 : 1.9186471700668335 

핵심키워드
인구 : 3.6806445121765137 
지역 : 2.842866897583008 
간 : 2.663395404815674 
이동 : 2.6485180854797363 
데이터 : 2.024946451187134 

핵심키워드
데이터 : 4.3792619705200195 
금융 : 4.025979995727539 
중계 : 3.579340934753418 
인프라 : 2.860461950302124 
개발 : 2.756382703781128 

